# Cases Data Processing

In [ ]:
import pandas as pd
import numpy as np
from glob import glob
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
feats = ['case_month',	'res_state',	'state_fips_code',	'res_county',	'county_fips_code',	
         'age_group',	'sex',	'race',	'ethnicity',	'case_positive_specimen_interval',	
         'case_onset_interval',	'process',	'exposure_yn',	'current_status',	'symptom_status',	
         'underlying_conditions_yn']
selected_feats = []
for feat in feats:
  feature = pd.read_csv("/content/drive/MyDrive/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv", usecols = [feat])
  feature = feature.replace(to_replace = ['Missing', 'Unknown'], value=np.nan)
  if feature.isnull().mean().values[0]>0.4:
    continue
  selected_feats.append(feat)

In [ ]:
selected_feats = ['case_month',
 'res_state',
 'state_fips_code',
 'res_county',
 'county_fips_code',
 'age_group',
 'sex',
 'race',
 'current_status']

In [ ]:
## Round 1

for j, dfgeo in enumerate(pd.read_csv("/content/drive/MyDrive/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv",
                    chunksize=1000000)):
  # replacing missing and unknown with null
  dfgeo_cleaned = dfgeo.replace(to_replace = ['Missing', 'Unknown', 'nul'], value=np.nan)

  # dropping rows with all targets being null
  mask = []
  target_vars = ['hosp_yn', 'icu_yn', 'death_yn']
  na_inds = dfgeo_cleaned.loc[dfgeo_cleaned[target_vars].isnull().sum(axis=1)==3,:].index
  dfgeo_cleaned.drop(na_inds, axis=0, inplace=True)

  # dropping columns with more than 40% missing
  dfgeo_cleaned.drop([col for col in dfgeo_cleaned.columns if col not in selected_feats and \
                      col not in target_vars], axis=1, inplace=True)

  # dropping rows with missing values in them
  missing_feats = [col for col in selected_feats if dfgeo_cleaned[col].isnull().sum()>0]
  df_tmp = dfgeo_cleaned.loc[:, missing_feats].dropna()

  dfgeo_cleaned = dfgeo_cleaned.loc[df_tmp.index, :]

  # saving to drive
  dfgeo_cleaned.to_csv(f'/content/drive/MyDrive/CSE6242 Clean Partitions/GEO_DATA/df_geo_partition_{j}.csv',
                      index=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
## Round 2

round1_files = glob('/content/drive/MyDrive/CSE6242 Clean Partitions/GEO_DATA/*.csv')

target_vars = ['hosp_yn', 'icu_yn', 'death_yn']
for j, file in enumerate(round1_files):
  dfgeo_r1 = pd.read_csv(file)

  # replacing case_month with numeric year and month columns
  if 'year' not in dfgeo_r1.columns:
    dfgeo_r1.insert(loc=0, column='year', value = dfgeo_r1['case_month'].apply(lambda x: x.split('-')[0]).astype('int') )
    dfgeo_r1.insert(loc=1, column='month', value= dfgeo_r1['case_month'].apply(lambda x: x.split('-')[1]).astype('int') )
    dfgeo_r1.drop('case_month', axis=1, inplace=True)

  # dropping state fips code (we'll use county fips code to join with VAX data)
  if 'state_fips_code' in dfgeo_r1.columns:
    dfgeo_r1.drop('state_fips_code', axis=1, inplace=True)

  # target variables binarization
  for var in target_vars:
    dfgeo_r1[var] = dfgeo_r1[var].replace({'Yes':1, 'No': 0})

  # over-writing cleaned frames
  dfgeo_r1.to_csv(file, index=False)

In [ ]:
## Round 2 Contd. Merging vax data on fips code

# get all unique fips codes from df_geo

fips_codes = pd.read_csv("/content/drive/MyDrive/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv", usecols=['county_fips_code'])
unique_fips = pd.Series(fips_codes.squeeze().unique(), name='county_fips_code')
vax_data = pd.read_csv("/content/drive/MyDrive/COVID-19_Vaccinations_in_the_United_States_County.csv", usecols = ['Date', 'FIPS',
                                                                                                                  'Completeness_pct',
                                                                                                                  'SVI_CTGY', 
                                                                                                                  'Metro_status', 
                                                                                                                  'Census2019'])


In [ ]:
vax_data = vax_data.loc[vax_data['FIPS']!='UNK', :]
vax_data['FIPS'] = vax_data['FIPS'].astype('int')
vax_data['Date'] = pd.to_datetime(vax_data['Date'], format='%m/%d/%Y')
vax_data['Metro_status'] = vax_data['Metro_status'].replace({'Non-metro':0, 'Metro': 1})
vax_data['SVI_CTGY'] = vax_data['SVI_CTGY'].replace({'D': 1, 'C': 2, 'B': 3, 'A': 4})

df3 = pd.merge(left=unique_fips, right=vax_data, how='inner', left_on='county_fips_code', right_on='FIPS')
shared_fips = [c for c in unique_fips if c in df3.FIPS]

In [ ]:
df3.head()

,county_fips_code,Date,FIPS,Completeness_pct,SVI_CTGY,Metro_status,Census2019
0,42091.0,2022-03-09,42091.0,94.7,4.0,1.0,830915.0
1,42091.0,2022-03-08,42091.0,94.7,4.0,1.0,830915.0
2,42091.0,2022-03-07,42091.0,94.7,4.0,1.0,830915.0
3,42091.0,2022-03-06,42091.0,94.7,4.0,1.0,830915.0
4,42091.0,2022-03-05,42091.0,94.7,4.0,1.0,830915.0


In [ ]:
# Round 2 - Contd.

# generating one-hot encoders for categorical features
round1_files = glob('/content/drive/MyDrive/CSE6242 Clean Partitions/GEO_DATA/*.csv')
catg_feats = ['res_state', 'age_group', 'sex', 'race', 'current_status']
catg_feat_encoders = {f: None for f in catg_feats}

# iterating over categorical features
for feat in catg_feats:
  # fit OHE for the specific feature at a time on all datapoints (original set)
  feature = pd.read_csv('/content/drive/MyDrive/COVID-19_Case_Surveillance_Public_Use_Data_with_Geography.csv', usecols=[feat])
  feature = feature.replace(to_replace = ['Missing', 'Unknown'], value=np.nan).dropna()

  catg_feat_encoders[feat] = OneHotEncoder(sparse=False).fit(feature)

In [ ]:
## Round 2 Contd. 

round1_files = glob('/content/drive/MyDrive/CSE6242 Clean Partitions/GEO_DATA/*.csv')
# iterate over all cleaned partitions to apply encoding
for j, file in enumerate(round1_files):
  dfgeo_r1 = pd.read_csv(file)
  dfgeo_r1.insert(loc=0, column = 'county_census_2019', value=np.nan)
  dfgeo_r1.insert(loc=0, column = 'county_SVI', value=np.nan)
  dfgeo_r1.insert(loc=0, column = 'county_metro_yn', value=np.nan)

  # instead of OHE county, use census, SVI and metro info from vax data to replace and drop county categorical variable
  for fips in shared_fips:
    dfgeo_r1.loc[dfgeo_r1.county_fips_code==fips, 'county_census_2019'] = list(df3.loc[df3.FIPS == fips, 'Census2019'])[0]
    dfgeo_r1.loc[dfgeo_r1.county_fips_code==fips, 'county_SVI'] = list(df3.loc[df3.FIPS == fips, 'SVI_CTGY'])[0]
    dfgeo_r1.loc[dfgeo_r1.county_fips_code==fips, 'county_metro_yn'] = list(df3.loc[df3.FIPS == fips, 'Metro_status'])[0]
  
  dfgeo_r1 = dfgeo_r1.drop(['res_county', 'county_fips_code'], axis=1)

  # One-hot encode catg feats in the file
  if dfgeo_r1.shape[0]>10:
    for feat in catg_feats:
      ohe_enc = catg_feat_encoders[feat].transform(dfgeo_r1.loc[:, [feat]])
      for i, catg in enumerate(catg_feat_encoders[feat].categories_[0]):
        dfgeo_r1.insert(loc=0, column = f'{feat}_{"_".join(catg.split())}', value= ohe_enc[:, i])
      dfgeo_r1.drop(feat, axis=1, inplace=True)
    dfgeo_r1.to_csv(f'/content/drive/MyDrive/CSE6242 Clean Partitions/df_partition_{j}.csv', index=False)

In [ ]:
files = glob('/content/drive/MyDrive/CSE6242 Clean Partitions/*.csv')

row_count = 0
col_count = 0

for i, file in enumerate(files):
  
  dfgeo_r1 = pd.read_csv(file)
  if i==0: col_count = dfgeo_r1.shape[1]
  row_count+= dfgeo_r1.shape[0]

In [ ]:
row_count

20265828

In [ ]:
col_count

77

# Processing Steps Taken

## Round 1

### 1.0 Replacing Unknowns and Missings with Null

### 1.1 Cleaning Rows with Missings or Unknowns

### 1.2 Dropping Rows with All targets null

### 1.3 Eliminating features with more than 40% missing

### 1.4 Missing Data Dropping (in feature columns)

## Round 2

### 2.0 Separating Date to features (Year and month)

### 2.1 Replacing county column with vax data's census population, SVI, and Metro status


### 2.2 One-Hot Encoding Categorical Features

### 2.3 Train-Test Splitting

### 2.4 Scaling Data (if necessary)


In [ ]:
## first method using KNN - left unused due to lack of non-missing features ##
# from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
# from sklearn.model_selection import train_test_split

# missing_feats = [col for col in dfgeo_cleaned.columns if dfgeo_cleaned[col].isnull().sum()>0 and col not in target_vars]
# nonmissing_feats = [col for col in dfgeo_cleaned.columns if dfgeo_cleaned[col].isnull().sum()==0 and col not in target_vars]

# for missing_feat in missing_feats:
#   if dfgeo_cleaned[missing_feat].dtype=='O':
#     knn = KNeighborsClassifier()
#   else:
#     knn = KNeighborsRegressor()
#   missing_mask = dfgeo_cleaned[missing_feat].isnull()
#   X_train = dfgeo_cleaned.loc[:, nonmissing_feats].drop(missing_mask, axis=0)
#   X_test = dfgeo_cleaned.loc[missing_mask, nonmissing_feats]
#   y_train = dfgeo_cleaned.loc[:, missing_feat].drop(missing_mask, axis=0)
    
#   knn.fit(X_train, y_train)
#   dfgeo_cleaned.loc[missing_mask, missing_feat] = knn.predict(X_test)